<a href="https://colab.research.google.com/github/ZzangJay/team_project/blob/kimjisan/221017_antibio_onehot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import pandas as pd
from tqdm.autonotebook import tqdm
from datetime import timedelta, datetime

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  This is separate from the ipykernel package so we can avoid doing imports until


In [3]:
pr_ab=pd.read_csv('/content/drive/MyDrive/mimiciv preprocessing/prescriptions_ab')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (10,14,16) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [4]:
icu_stay=pd.read_csv('/content/drive/MyDrive/mimiciv_unzip/icu/icustays.csv')

In [5]:
micro_events=pd.read_csv('/content/drive/MyDrive/mimiciv_unzip/hosp/microbiologyevents.csv')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (16) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [6]:
icu_stay

,subject_id,hadm_id,stay_id,first_careunit,last_careunit,intime,outtime,los
0,10000032,29079034,39553978,Medical Intensive Care Unit (MICU),Medical Intensive Care Unit (MICU),2180-07-23 14:00:00,2180-07-23 23:50:47,0.410266
1,10000980,26913865,39765666,Medical Intensive Care Unit (MICU),Medical Intensive Care Unit (MICU),2189-06-27 08:42:00,2189-06-27 20:38:27,0.497535
2,10001217,24597018,37067082,Surgical Intensive Care Unit (SICU),Surgical Intensive Care Unit (SICU),2157-11-20 19:18:02,2157-11-21 22:08:00,1.118032
3,10001217,27703517,34592300,Surgical Intensive Care Unit (SICU),Surgical Intensive Care Unit (SICU),2157-12-19 15:42:24,2157-12-20 14:27:41,0.948113
4,10001725,25563031,31205490,Medical/Surgical Intensive Care Unit (MICU/SICU),Medical/Surgical Intensive Care Unit (MICU/SICU),2110-04-11 15:52:22,2110-04-12 23:59:56,1.338588
...,...,...,...,...,...,...,...,...
76938,19999442,26785317,32336619,Surgical Intensive Care Unit (SICU),Surgical Intensive Care Unit (SICU),2148-11-19 14:23:43,2148-11-26 13:12:15,6.950370
76939,19999625,25304202,31070865,Medical/Surgical Intensive Care Unit (MICU/SICU),Medical/Surgical Intensive Care Unit (MICU/SICU),2139-10-10 19:18:00,2139-10-11 18:21:28,0.960741
76940,19999828,25744818,36075953,Medical Intensive Care Unit (MICU),Medical Intensive Care Unit (MICU),2149-01-08 18:12:00,2149-01-10 13:11:02,1.790995
76941,19999840,21033226,38978960,Trauma SICU (TSICU),Surgical Intensive Care Unit (SICU),2164-09-12 09:26:28,2164-09-17 16:35:15,5.297766


In [7]:
icu_time=icu_stay[['subject_id','hadm_id','stay_id','intime', 'outtime']]

In [8]:
ab_time=pr_ab[['hadm_id','drug','starttime','stoptime']]

In [9]:
icu_time.columns=['subject_id','hadm_id','stay_id','icu_intime', 'icu_outtime']

In [10]:
icu_time

,subject_id,hadm_id,stay_id,icu_intime,icu_outtime
0,10000032,29079034,39553978,2180-07-23 14:00:00,2180-07-23 23:50:47
1,10000980,26913865,39765666,2189-06-27 08:42:00,2189-06-27 20:38:27
2,10001217,24597018,37067082,2157-11-20 19:18:02,2157-11-21 22:08:00
3,10001217,27703517,34592300,2157-12-19 15:42:24,2157-12-20 14:27:41
4,10001725,25563031,31205490,2110-04-11 15:52:22,2110-04-12 23:59:56
...,...,...,...,...,...
76938,19999442,26785317,32336619,2148-11-19 14:23:43,2148-11-26 13:12:15
76939,19999625,25304202,31070865,2139-10-10 19:18:00,2139-10-11 18:21:28
76940,19999828,25744818,36075953,2149-01-08 18:12:00,2149-01-10 13:11:02
76941,19999840,21033226,38978960,2164-09-12 09:26:28,2164-09-17 16:35:15


In [11]:
ab_time.columns=['hadm_id','drug','ab_starttime','ab_stoptime']

In [12]:
ab_time

,hadm_id,drug,ab_starttime,ab_stoptime
0,22841357,sulfameth/trimethoprim ds,2180-06-26 23:00:00,2180-06-27 23:00:00
1,27988844,cefazolin,2183-09-19 12:00:00,2183-09-20 15:00:00
2,20823482,cefazolin,2145-04-25 00:00:00,2145-04-28 20:00:00
3,27897940,amoxicillin-clavulanic acid,2132-10-15 20:00:00,2132-10-19 00:00:00
4,20032235,ciprofloxacin hcl,2146-12-09 10:00:00,2146-12-12 13:00:00
...,...,...,...,...
678034,21033226,piperacillin-tazobactam na,2164-09-15 12:00:00,2164-09-16 12:00:00
678035,26071774,sulfameth/trimethoprim ds,2164-07-26 20:00:00,2164-07-26 13:00:00
678036,23865745,vancomycin,2145-11-08 20:00:00,2145-11-09 15:00:00
678037,23865745,sulfameth/trimethoprim ds,2145-11-09 20:00:00,2145-11-11 17:00:00


In [13]:
icu_ab_join = pd.merge(icu_time, ab_time, left_on='hadm_id', right_on='hadm_id', how='left')

In [14]:
icu_ab_join

,subject_id,hadm_id,stay_id,icu_intime,icu_outtime,drug,ab_starttime,ab_stoptime
0,10000032,29079034,39553978,2180-07-23 14:00:00,2180-07-23 23:50:47,NaN,NaN,NaN
1,10000980,26913865,39765666,2189-06-27 08:42:00,2189-06-27 20:38:27,levofloxacin,2189-06-27 11:00:00,2189-06-28 11:00:00
2,10001217,24597018,37067082,2157-11-20 19:18:02,2157-11-21 22:08:00,vancomycin,2157-11-22 04:00:00,2157-11-23 10:00:00
3,10001217,24597018,37067082,2157-11-20 19:18:02,2157-11-21 22:08:00,metronidazole (flagyl),2157-11-23 10:00:00,2157-11-25 22:00:00
4,10001217,24597018,37067082,2157-11-20 19:18:02,2157-11-21 22:08:00,vancomycin,2157-11-20 19:00:00,2157-11-22 02:00:00
...,...,...,...,...,...,...,...,...
394120,19999840,21033226,38978960,2164-09-12 09:26:28,2164-09-17 16:35:15,vancomycin,2164-09-15 20:00:00,2164-09-16 12:00:00
394121,19999840,21033226,38978960,2164-09-12 09:26:28,2164-09-17 16:35:15,piperacillin-tazobactam na,2164-09-15 12:00:00,2164-09-16 12:00:00
394122,19999987,23865745,36195440,2145-11-02 22:59:00,2145-11-04 21:29:30,vancomycin,2145-11-08 20:00:00,2145-11-09 15:00:00
394123,19999987,23865745,36195440,2145-11-02 22:59:00,2145-11-04 21:29:30,sulfameth/trimethoprim ds,2145-11-09 20:00:00,2145-11-11 17:00:00


In [15]:
icu_ab_join['drug'].isna().sum()

21329

In [16]:
icu_ab_join.dropna(subset=['drug','ab_starttime','ab_stoptime'], axis=0,inplace=True)

In [17]:
icu_ab_join

,subject_id,hadm_id,stay_id,icu_intime,icu_outtime,drug,ab_starttime,ab_stoptime
1,10000980,26913865,39765666,2189-06-27 08:42:00,2189-06-27 20:38:27,levofloxacin,2189-06-27 11:00:00,2189-06-28 11:00:00
2,10001217,24597018,37067082,2157-11-20 19:18:02,2157-11-21 22:08:00,vancomycin,2157-11-22 04:00:00,2157-11-23 10:00:00
3,10001217,24597018,37067082,2157-11-20 19:18:02,2157-11-21 22:08:00,metronidazole (flagyl),2157-11-23 10:00:00,2157-11-25 22:00:00
4,10001217,24597018,37067082,2157-11-20 19:18:02,2157-11-21 22:08:00,vancomycin,2157-11-20 19:00:00,2157-11-22 02:00:00
5,10001217,27703517,34592300,2157-12-19 15:42:24,2157-12-20 14:27:41,vancomycin,2157-12-23 09:00:00,2157-12-24 19:00:00
...,...,...,...,...,...,...,...,...
394120,19999840,21033226,38978960,2164-09-12 09:26:28,2164-09-17 16:35:15,vancomycin,2164-09-15 20:00:00,2164-09-16 12:00:00
394121,19999840,21033226,38978960,2164-09-12 09:26:28,2164-09-17 16:35:15,piperacillin-tazobactam na,2164-09-15 12:00:00,2164-09-16 12:00:00
394122,19999987,23865745,36195440,2145-11-02 22:59:00,2145-11-04 21:29:30,vancomycin,2145-11-08 20:00:00,2145-11-09 15:00:00
394123,19999987,23865745,36195440,2145-11-02 22:59:00,2145-11-04 21:29:30,sulfameth/trimethoprim ds,2145-11-09 20:00:00,2145-11-11 17:00:00


In [18]:
icu_ab_join.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 371251 entries, 1 to 394124
Data columns (total 8 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   subject_id    371251 non-null  int64 
 1   hadm_id       371251 non-null  int64 
 2   stay_id       371251 non-null  int64 
 3   icu_intime    371251 non-null  object
 4   icu_outtime   371251 non-null  object
 5   drug          371251 non-null  object
 6   ab_starttime  371251 non-null  object
 7   ab_stoptime   371251 non-null  object
dtypes: int64(3), object(5)
memory usage: 25.5+ MB


In [19]:
icu_ab_join.to_csv('/content/drive/MyDrive/mimiciv preprocessing/icu_ab_join.csv',index=False)

In [20]:
icu_path = '/content/drive/MyDrive/mimiciv_unzip/icu/'
new_path = '/content/drive/MyDrive/mimiciv preprocessing/'
hos_path = '/content/drive/MyDrive/mimiciv_unzip/hosp/'

In [21]:
import pickle
with open(new_path+'adm_stay.pkl','rb') as f:
    dict_ = pickle.load(f)


admissions + icustay 불러오기 (hadm_id기준 labeling 한 데이터)

In [22]:
stay_dict=dict_

In [23]:
dict_

{24903681: [{'INTIME': numpy.datetime64('2132-11-05T11:36:59.000000000'),
   'OUTTIME': numpy.datetime64('2132-11-08T01:26:26.000000000'),
   'IS_DEAD': 0,
   'DOD': numpy.datetime64('NaT'),
   'STAYID': 36685455}],
 24510466: [{'INTIME': numpy.datetime64('2185-07-04T22:43:50.000000000'),
   'OUTTIME': numpy.datetime64('2185-07-08T14:54:12.000000000'),
   'IS_DEAD': 1,
   'DOD': numpy.datetime64('2185-07-08T09:35:00.000000000'),
   'STAYID': 33539556}],
 27262979: [{'INTIME': numpy.datetime64('2154-02-04T09:04:00.000000000'),
   'OUTTIME': numpy.datetime64('2154-02-05T19:33:30.000000000'),
   'IS_DEAD': 0,
   'DOD': numpy.datetime64('NaT'),
   'STAYID': 35611408}],
 25559044: [{'INTIME': numpy.datetime64('2146-05-15T13:57:06.000000000'),
   'OUTTIME': numpy.datetime64('2146-05-17T11:58:29.000000000'),
   'IS_DEAD': 0,
   'DOD': numpy.datetime64('NaT'),
   'STAYID': 32167668}],
 25034757: [{'INTIME': numpy.datetime64('2160-05-15T17:38:35.000000000'),
   'OUTTIME': numpy.datetime64('2160

In [24]:
stay_list=[]
for v in (list(dict_.values())):
  stay_list.append(v[0]['STAYID'])

In [25]:
len(stay_list)

69639

In [26]:
icu_ab_st=icu_ab_join[icu_ab_join['stay_id'].isin(stay_list)]

In [27]:
stay_list_js=icu_ab_join['stay_id'].unique()

In [28]:
len(set(stay_list))

69639

In [29]:
len(set(stay_list)&set(stay_list_js))

48989

In [30]:
stay_list_both=list(set(stay_list)&set(stay_list_js))

In [31]:
icu_ab_both=icu_ab_join[icu_ab_join['stay_id'].isin(stay_list_both)]

In [32]:
icu_ab_both

,subject_id,hadm_id,stay_id,icu_intime,icu_outtime,drug,ab_starttime,ab_stoptime
1,10000980,26913865,39765666,2189-06-27 08:42:00,2189-06-27 20:38:27,levofloxacin,2189-06-27 11:00:00,2189-06-28 11:00:00
2,10001217,24597018,37067082,2157-11-20 19:18:02,2157-11-21 22:08:00,vancomycin,2157-11-22 04:00:00,2157-11-23 10:00:00
3,10001217,24597018,37067082,2157-11-20 19:18:02,2157-11-21 22:08:00,metronidazole (flagyl),2157-11-23 10:00:00,2157-11-25 22:00:00
4,10001217,24597018,37067082,2157-11-20 19:18:02,2157-11-21 22:08:00,vancomycin,2157-11-20 19:00:00,2157-11-22 02:00:00
5,10001217,27703517,34592300,2157-12-19 15:42:24,2157-12-20 14:27:41,vancomycin,2157-12-23 09:00:00,2157-12-24 19:00:00
...,...,...,...,...,...,...,...,...
394120,19999840,21033226,38978960,2164-09-12 09:26:28,2164-09-17 16:35:15,vancomycin,2164-09-15 20:00:00,2164-09-16 12:00:00
394121,19999840,21033226,38978960,2164-09-12 09:26:28,2164-09-17 16:35:15,piperacillin-tazobactam na,2164-09-15 12:00:00,2164-09-16 12:00:00
394122,19999987,23865745,36195440,2145-11-02 22:59:00,2145-11-04 21:29:30,vancomycin,2145-11-08 20:00:00,2145-11-09 15:00:00
394123,19999987,23865745,36195440,2145-11-02 22:59:00,2145-11-04 21:29:30,sulfameth/trimethoprim ds,2145-11-09 20:00:00,2145-11-11 17:00:00


In [33]:
icu_ab_both=icu_ab_both.sort_values(by=['stay_id','ab_starttime'])

In [34]:
icu_ab_both

,subject_id,hadm_id,stay_id,icu_intime,icu_outtime,drug,ab_starttime,ab_stoptime
99007,12466550,23998182,30000153,2174-09-29 12:09:00,2174-10-01 03:26:10,piperacillin-tazobactam na,2174-10-04 10:00:00,2174-10-04 19:00:00
98998,12466550,23998182,30000153,2174-09-29 12:09:00,2174-10-01 03:26:10,piperacillin-tazobactam na,2174-10-04 18:00:00,2174-10-04 19:00:00
98997,12466550,23998182,30000153,2174-09-29 12:09:00,2174-10-01 03:26:10,vancomycin,2174-10-04 20:00:00,2174-10-07 11:00:00
99002,12466550,23998182,30000153,2174-09-29 12:09:00,2174-10-01 03:26:10,vancomycin,2174-10-04 20:00:00,2174-10-04 19:00:00
99003,12466550,23998182,30000153,2174-09-29 12:09:00,2174-10-01 03:26:10,piperacillin-tazobactam na,2174-10-04 20:00:00,2174-10-07 11:00:00
...,...,...,...,...,...,...,...,...
246247,16180713,20178444,39999301,2111-08-18 15:47:00,2111-08-19 19:18:33,erythromycin,2111-08-19 14:00:00,2111-08-19 23:00:00
218202,15498623,27161460,39999384,2158-05-24 19:50:01,2158-05-26 02:27:07,cefazolin,2158-05-24 21:00:00,2158-05-25 03:00:00
51076,11256534,26910235,39999552,2186-07-17 14:27:43,2186-07-18 18:29:05,cefazolin,2186-07-17 14:00:00,2186-07-18 18:00:00
51077,11256534,26910235,39999552,2186-07-17 14:27:43,2186-07-18 18:29:05,cefazolin,2186-07-18 18:00:00,2186-07-20 02:00:00


In [35]:
icu_ab_both=icu_ab_both.drop_duplicates(subset=['stay_id'], keep='first', inplace=False, ignore_index=False)

In [36]:
icu_ab_both

,subject_id,hadm_id,stay_id,icu_intime,icu_outtime,drug,ab_starttime,ab_stoptime
99007,12466550,23998182,30000153,2174-09-29 12:09:00,2174-10-01 03:26:10,piperacillin-tazobactam na,2174-10-04 10:00:00,2174-10-04 19:00:00
332964,18421337,22413411,30000484,2136-01-14 17:23:32,2136-01-17 04:53:08,levofloxacin,2136-01-14 21:00:00,2136-01-15 16:00:00
89502,12207593,22795209,30000646,2194-04-29 01:39:22,2194-05-03 18:23:48,vancomycin,2194-04-29 07:00:00,2194-04-30 06:00:00
118421,12980335,23552849,30001148,2156-08-30 11:10:59,2156-08-31 14:25:34,vancomycin,2156-08-30 20:00:00,2156-08-31 15:00:00
291967,17371178,24502166,30001396,2147-10-18 12:48:00,2147-10-19 16:49:23,vancomycin,2147-10-20 07:00:00,2147-10-21 20:00:00
...,...,...,...,...,...,...,...,...
53287,11305179,22341980,39999286,2158-03-07 11:15:18,2158-03-08 15:02:06,ampicillin-sulbactam,2158-03-05 11:00:00,2158-03-06 11:00:00
246247,16180713,20178444,39999301,2111-08-18 15:47:00,2111-08-19 19:18:33,erythromycin,2111-08-19 14:00:00,2111-08-19 23:00:00
218202,15498623,27161460,39999384,2158-05-24 19:50:01,2158-05-26 02:27:07,cefazolin,2158-05-24 21:00:00,2158-05-25 03:00:00
51076,11256534,26910235,39999552,2186-07-17 14:27:43,2186-07-18 18:29:05,cefazolin,2186-07-17 14:00:00,2186-07-18 18:00:00


In [37]:
icu_ab_both['drug'].value_counts()

cefazolin                             11267
vancomycin                            11033
piperacillin-tazobactam                3866
cefepime                               3556
ciprofloxacin hcl                      2237
                                      ...  
piperacillin-tazo graded challenge        1
zithromax z-pak                           1
penicillin g k graded challenge           1
clindamycin phosphate                     1
chloramphenicol na succ                   1
Name: drug, Length: 67, dtype: int64

In [38]:
ab_class=pd.read_csv('/content/drive/MyDrive/mimiciv preprocessing/antibiotics_Class.csv')

In [39]:
ab_class['Class'].unique()

array(['aminoglycosides', 'cephalosporins', 'chroramphenicol',
       'fluoroquinolones', 'glycopeptides', 'licosamides(lincomycin)',
       'macrolides', 'monobactams', 'nitrofuran(toin)', 'nitroimidazole ',
       'penicillins', 'rifamycins', 'sulfonamides(diaminopyrimidine)',
       'tetracyclines'], dtype=object)

In [40]:
for i in tqdm(icu_ab_both['drug'],desc='convert',total=48989):
  for j in range(len(ab_class['Antibiotics'])):
    if i == ab_class['Antibiotics'][j] :
       icu_ab_both['drug'].replace(i,ab_class['Class'][j],inplace=True)
    else : pass

convert:   0%|          | 0/48989 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:6619: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)


In [41]:
icu_ab_both['drug'].value_counts()

cephalosporins                     15820
glycopeptides                      11608
penicillins                         6268
fluoroquinolones                    5739
macrolides                          3020
nitroimidazole                      2753
sulfonamides(diaminopyrimidine)     1779
licosamides(lincomycin)              734
tetracyclines                        473
aminoglycosides                      350
nitrofuran(toin)                     288
monobactams                          116
rifamycins                            40
chroramphenicol                        1
Name: drug, dtype: int64

In [56]:
icu_ab_1=icu_ab_both[icu_ab_both['drug'].isin(['aminoglycosides','nitrofuran(toin)','monobactams','rifamycins','sulfonamides','chroramphenicol'])==False]

In [43]:
icu_ab_1

,subject_id,hadm_id,stay_id,icu_intime,icu_outtime,drug,ab_starttime,ab_stoptime
99007,12466550,23998182,30000153,2174-09-29 12:09:00,2174-10-01 03:26:10,penicillins,2174-10-04 10:00:00,2174-10-04 19:00:00
332964,18421337,22413411,30000484,2136-01-14 17:23:32,2136-01-17 04:53:08,fluoroquinolones,2136-01-14 21:00:00,2136-01-15 16:00:00
89502,12207593,22795209,30000646,2194-04-29 01:39:22,2194-05-03 18:23:48,glycopeptides,2194-04-29 07:00:00,2194-04-30 06:00:00
118421,12980335,23552849,30001148,2156-08-30 11:10:59,2156-08-31 14:25:34,glycopeptides,2156-08-30 20:00:00,2156-08-31 15:00:00
291967,17371178,24502166,30001396,2147-10-18 12:48:00,2147-10-19 16:49:23,glycopeptides,2147-10-20 07:00:00,2147-10-21 20:00:00
...,...,...,...,...,...,...,...,...
53287,11305179,22341980,39999286,2158-03-07 11:15:18,2158-03-08 15:02:06,penicillins,2158-03-05 11:00:00,2158-03-06 11:00:00
246247,16180713,20178444,39999301,2111-08-18 15:47:00,2111-08-19 19:18:33,macrolides,2111-08-19 14:00:00,2111-08-19 23:00:00
218202,15498623,27161460,39999384,2158-05-24 19:50:01,2158-05-26 02:27:07,cephalosporins,2158-05-24 21:00:00,2158-05-25 03:00:00
51076,11256534,26910235,39999552,2186-07-17 14:27:43,2186-07-18 18:29:05,cephalosporins,2186-07-17 14:00:00,2186-07-18 18:00:00


In [57]:
icu_ab_1['drug'].value_counts()

cephalosporins                     15820
glycopeptides                      11608
penicillins                         6268
fluoroquinolones                    5739
macrolides                          3020
nitroimidazole                      2753
sulfonamides(diaminopyrimidine)     1779
licosamides(lincomycin)              734
tetracyclines                        473
Name: drug, dtype: int64

In [58]:
icu_ab_1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 48194 entries, 99007 to 214250
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   subject_id    48194 non-null  int64 
 1   hadm_id       48194 non-null  int64 
 2   stay_id       48194 non-null  int64 
 3   icu_intime    48194 non-null  object
 4   icu_outtime   48194 non-null  object
 5   drug          48194 non-null  object
 6   ab_starttime  48194 non-null  object
 7   ab_stoptime   48194 non-null  object
dtypes: int64(3), object(5)
memory usage: 3.3+ MB


In [53]:
from datetime import datetime

NOT_CONVERTED = 'NOT_CONVERTED'


def str2datetime(s):
    def _convert(_s, _dformat):
        try:
            converted_dt = datetime.strptime(_s, _dformat)
        except Exception:
            return NOT_CONVERTED

        return converted_dt

    if isinstance(s, datetime):
        return s

    dformats = [
        '%Y-%m-%d %p %I:%M:%S',
        '%Y-%m-%d %H:%M:%S.%f',
        '%Y-%m-%d %H:%M:%S',
        '%Y-%m-%d %H:%M',
        '%y-%m-%d %H:%M',
        '%Y-%m-%d',
        '%Y%m%d',
        '%Y-%m-%d +%H:%M',
        '%Y%m%d %H%M',
        '%Y%m%d%H%M%S',
        ]

    s = s.split('.')[0]
    for dformat in dformats:
        dt = _convert(s, dformat)
        if dt != NOT_CONVERTED:
            return dt

    return NOT_CONVERTED

In [59]:
timelist=['icu_intime','icu_outtime','ab_starttime','ab_stoptime']
for col in timelist:
  for i in tqdm(icu_ab_1[col],desc='convert_dt',total=48194):
    icu_ab_1[col].replace(i,str2datetime(i),inplace=True)

convert_dt:   0%|          | 0/48194 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:6619: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)


convert_dt:   0%|          | 0/48194 [00:00<?, ?it/s]

convert_dt:   0%|          | 0/48194 [00:00<?, ?it/s]

convert_dt:   0%|          | 0/48194 [00:00<?, ?it/s]

In [60]:
icu_ab_1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 48194 entries, 99007 to 214250
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   subject_id    48194 non-null  int64         
 1   hadm_id       48194 non-null  int64         
 2   stay_id       48194 non-null  int64         
 3   icu_intime    48194 non-null  datetime64[ns]
 4   icu_outtime   48194 non-null  datetime64[ns]
 5   drug          48194 non-null  object        
 6   ab_starttime  48194 non-null  datetime64[ns]
 7   ab_stoptime   48194 non-null  datetime64[ns]
dtypes: datetime64[ns](4), int64(3), object(1)
memory usage: 3.3+ MB


In [61]:
icu_ab_1

,subject_id,hadm_id,stay_id,icu_intime,icu_outtime,drug,ab_starttime,ab_stoptime
99007,12466550,23998182,30000153,2174-09-29 12:09:00,2174-10-01 03:26:10,penicillins,2174-10-04 10:00:00,2174-10-04 19:00:00
332964,18421337,22413411,30000484,2136-01-14 17:23:32,2136-01-17 04:53:08,fluoroquinolones,2136-01-14 21:00:00,2136-01-15 16:00:00
89502,12207593,22795209,30000646,2194-04-29 01:39:22,2194-05-03 18:23:48,glycopeptides,2194-04-29 07:00:00,2194-04-30 06:00:00
118421,12980335,23552849,30001148,2156-08-30 11:10:59,2156-08-31 14:25:34,glycopeptides,2156-08-30 20:00:00,2156-08-31 15:00:00
291967,17371178,24502166,30001396,2147-10-18 12:48:00,2147-10-19 16:49:23,glycopeptides,2147-10-20 07:00:00,2147-10-21 20:00:00
...,...,...,...,...,...,...,...,...
53287,11305179,22341980,39999286,2158-03-07 11:15:18,2158-03-08 15:02:06,penicillins,2158-03-05 11:00:00,2158-03-06 11:00:00
246247,16180713,20178444,39999301,2111-08-18 15:47:00,2111-08-19 19:18:33,macrolides,2111-08-19 14:00:00,2111-08-19 23:00:00
218202,15498623,27161460,39999384,2158-05-24 19:50:01,2158-05-26 02:27:07,cephalosporins,2158-05-24 21:00:00,2158-05-25 03:00:00
51076,11256534,26910235,39999552,2186-07-17 14:27:43,2186-07-18 18:29:05,cephalosporins,2186-07-17 14:00:00,2186-07-18 18:00:00


In [66]:
icu_ab_1.to_csv('/content/drive/MyDrive/mimiciv preprocessing/antibio_nonenc.csv')

In [62]:
icu_ab_onehot=pd.get_dummies(icu_ab_1)

In [63]:
icu_ab_onehot

,subject_id,hadm_id,stay_id,icu_intime,icu_outtime,ab_starttime,ab_stoptime,drug_cephalosporins,drug_fluoroquinolones,drug_glycopeptides,drug_licosamides(lincomycin),drug_macrolides,drug_nitroimidazole,drug_penicillins,drug_sulfonamides(diaminopyrimidine),drug_tetracyclines
99007,12466550,23998182,30000153,2174-09-29 12:09:00,2174-10-01 03:26:10,2174-10-04 10:00:00,2174-10-04 19:00:00,0,0,0,0,0,0,1,0,0
332964,18421337,22413411,30000484,2136-01-14 17:23:32,2136-01-17 04:53:08,2136-01-14 21:00:00,2136-01-15 16:00:00,0,1,0,0,0,0,0,0,0
89502,12207593,22795209,30000646,2194-04-29 01:39:22,2194-05-03 18:23:48,2194-04-29 07:00:00,2194-04-30 06:00:00,0,0,1,0,0,0,0,0,0
118421,12980335,23552849,30001148,2156-08-30 11:10:59,2156-08-31 14:25:34,2156-08-30 20:00:00,2156-08-31 15:00:00,0,0,1,0,0,0,0,0,0
291967,17371178,24502166,30001396,2147-10-18 12:48:00,2147-10-19 16:49:23,2147-10-20 07:00:00,2147-10-21 20:00:00,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53287,11305179,22341980,39999286,2158-03-07 11:15:18,2158-03-08 15:02:06,2158-03-05 11:00:00,2158-03-06 11:00:00,0,0,0,0,0,0,1,0,0
246247,16180713,20178444,39999301,2111-08-18 15:47:00,2111-08-19 19:18:33,2111-08-19 14:00:00,2111-08-19 23:00:00,0,0,0,0,1,0,0,0,0
218202,15498623,27161460,39999384,2158-05-24 19:50:01,2158-05-26 02:27:07,2158-05-24 21:00:00,2158-05-25 03:00:00,1,0,0,0,0,0,0,0,0
51076,11256534,26910235,39999552,2186-07-17 14:27:43,2186-07-18 18:29:05,2186-07-17 14:00:00,2186-07-18 18:00:00,1,0,0,0,0,0,0,0,0


In [64]:
icu_ab_onehot.to_csv('/content/drive/MyDrive/mimiciv preprocessing/antibio_onehot.csv')

In [65]:
icu_ab_onehot.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 48194 entries, 99007 to 214250
Data columns (total 16 columns):
 #   Column                                Non-Null Count  Dtype         
---  ------                                --------------  -----         
 0   subject_id                            48194 non-null  int64         
 1   hadm_id                               48194 non-null  int64         
 2   stay_id                               48194 non-null  int64         
 3   icu_intime                            48194 non-null  datetime64[ns]
 4   icu_outtime                           48194 non-null  datetime64[ns]
 5   ab_starttime                          48194 non-null  datetime64[ns]
 6   ab_stoptime                           48194 non-null  datetime64[ns]
 7   drug_cephalosporins                   48194 non-null  uint8         
 8   drug_fluoroquinolones                 48194 non-null  uint8         
 9   drug_glycopeptides                    48194 non-null  uint8        